In [50]:
import numpy as np

from utilities.elements_utils import *
from Extractor.ext_util import *
from Extractor.extract import params
from visualizer.plot_funcs import *

# a = gdstk.read_gds("outputs/nmos_cm_6x3_2.gds")
# a = gdstk.read_gds("outputs/nmos_dp_2x6_0.gds")
a = gdstk.read_gds("outputs/nmos_dp_6x5_2.gds")

a1 = a.cells[0]
# Get important polygons from the gds file
polygons = get_polygons(a1)
nets = get_nets(polygons)
# show_layout(a1,fig_size=(2.5, 1.2))
# print(polygons["pins"][2].points)
# print(polygons["pins"][2].text)
# plot_polygons(polygons["metal_1"]+polygons["metal_2"],xlim=[0,2000],ylim=[0,2000])
# plot_polygons(polygons["active_areas"],xlim=[-500,2000],ylim=[-500,2000])
# print(a_a[0].points)
# plot_polygons(nets[2].metals, label=nets[2].label, xlim=[-500,2000],ylim=[-500,2000])
# plot_polygons(nets[1].metals, label=nets[1].label, xlim=[-500,2000],ylim=[-500,2000])
plot_nets(nets,2800)
# for i, net in enumerate(nets):
#     print(f"{i}   {net.id}")
#     
# for p in polygons["rvt"]:
#     print(p.id)


# for net in nets:
#     for cb in net.cbs:
#         print(f"{net.id} - {cb.mid_point}")
#       
# print(nets[1].id, len(nets[1].diffs))  

In [51]:
from Extractor.extract import *
for target_net in nets:
    js, _ = convert_net_to_stick(target_net, th1=50, th2=20) 
    nodes = find_all_nodes(js, target_net, node_reduction=True)

    fig, axes = plt.subplots(1, 4, figsize=(12, 4))  # Create a figure with 2 subplots
    show_gds(axes[0], a1)
    show_nets(axes[1], nets)
    show_nets(axes[2], [target_net])
    show_stk_nets(axes[3], js, nodes)
    plt.tight_layout()
    plt.show()
    plt.savefig("Extractor/img_"+str(target_net.id[0])+".png")
    plt.close()

target_net = nets[2]
js, _ = convert_net_to_stick(target_net, th1=50, th2=20) 
nodes = find_all_nodes(js, target_net, node_reduction=True)

In [74]:
graph = np.empty_like(nodes)
for i, nd in enumerate(nodes):
    adj = adjacent_nodes(nd, nodes, js)
    graph[i] = adj
def dfs_iterative(graph, start):
    visited = set()
    stack = [start]
    res = []
    while stack:
        vertex = stack.pop()
        if vertex not in visited:
            visited.add(vertex)
            neighbors = []
            for neighbor in graph[vertex]:
                if neighbor not in visited:
                    neighbors.append(neighbor)
            stack.extend(neighbors)
            res.append([vertex, neighbors])
            # stack.extend(neighbor for neighbor in graph[vertex] if neighbor not in visited)
    return res
label_index = 0
for i, nd in enumerate(nodes):
    if nd["t"] == "L":
        label_index = i
        break
nds = dfs_iterative(graph, label_index)

print(nds)

In [75]:
poly_counter = 0
diff_counter = 0

# giving ids for all polys and diffs
for n in nets:
    for poly in n.polys:
        poly.id = poly_counter
        
        for r in polygons["rvt"]:
            if are_they_connected(r, poly):
                poly.vt_box.append(r.id) 
                
        poly_counter += 1
    for diff in n.diffs:
        diff.id = diff_counter
        diff_counter += 1
        
# ------------------------------------------------
print("Polys")
for n in nets:
   for p in n.polys:
    print(p.id,f"{p.mid}  ", end=" ")

print("\nNets")
for n in nets:
   for d in n.diffs:
    print(d.id, end=" ")
# ------------------------------------------------

In [128]:
instances = {}  # np.empty([1,total_poly_no])
for n in nets:
    for poly in n.polys:
        vt_ind = poly.vt_box[0]
        vt_polygon = polygons["rvt"][vt_ind]
        r_point = [poly.mid + 78 / 2, vt_polygon.mid_point[1]]  # poly pitch/2
        l_point = [poly.mid - 78 / 2, vt_polygon.mid_point[1]]  # poly pitch/2
        # search for diffs
        instance_i = [-1, -1]
        for net in nets:
            for diff in net.diffs:
                # print(f"-- {l_point}, {diff.mid} --")
                if is_point_inside_the_polygon(l_point, diff):
                    instance_i[0] = diff.id
                if is_point_inside_the_polygon(r_point, diff):
                    instance_i[1] = diff.id
        
        instances[poly.id] = instance_i

# -------------------------------------------------
print(instances)
# -------------------------------------------------

In [138]:
# identifying instances with shared diffusion
class Diff:
    def __init__(self):
        self.net = -1
        self.instances = []
        self.type = ''
        self.side = ''
        
diff_array = {}  # diffs, associated_instances, net_name
for key in instances:
    inst = instances[key]
    l_diff = inst[0]
    r_diff = inst[1]
    l_diff_i = Diff()
    r_diff_i = Diff()
    if not l_diff in diff_array:
        l_diff_i.instances.append(key)
        l_diff_i.side = 'l'
        diff_array[l_diff] = l_diff_i
    else:
        diff_array[l_diff].side = 'b'
        diff_array[l_diff].instances.append(key)
    if not r_diff in diff_array:
        r_diff_i.instances.append(key)
        r_diff_i.side = 'r'
        diff_array[r_diff] = r_diff_i
    else:
        diff_array[r_diff].side = 'b'
        diff_array[r_diff].instances.append(key)
        
# ----------------------------------------------------------
for key, df in diff_array.items():
    print(key,"\t",df.instances,"\t", df.side)
# ----------------------------------------------------------

In [139]:
# assigning nets to the diffusions
for n in nets:
    for diff in n.diffs:
        if diff.id in diff_array:
            diff_array[diff.id].net = n.id[0]


# ----------------------------------------------------------
for key, df in diff_array.items():
    print(key,"\t",df.instances,"\t", df.side,"\t", df.net)
# ---------------------------------------------------------- 

In [140]:
# identifying the first source and drain diffs
visited = set()
for key in diff_array:
    if diff_array[key].side == 'l' and len(diff_array[key].instances) == 1:
        diff_array[key].type = 's'
        for k in instances:
            ins = instances[k]
            if ins[0] == key:
                diff_array[ins[1]].type = 'd'
                visited.add(ins[0])
                visited.add(ins[1])   
# # ----------------------------------------------------------
# for key, df in diff_array.items():
#     print(key,"\t",df.type,"\t",df.side,"\t", df.net)
#     
# print(visited)
# # ---------------------------------------------------------- 

t = ["s", "d"]
while len(visited) != len(diff_array):
    for k, ins in instances.items():
        if len(visited.intersection(ins))==1:
            # print("visited",visited, "ins", ins, len(visited.intersection(ins)))
            a = visited.intersection(ins)
            i = ins.index(list(a)[0])  
            # print("intersection:",a, "index", i)
            diff_array[ins[i^1]].type = t[t.index(diff_array[ins[i]].type)^1]
            visited.update(ins)                   

# # ----------------------------------------------------------
# for key, df in diff_array.items():
#     print(key,"\t",df.type,"\t",df.side,"\t", df.net)
# # ---------------------------------------------------------- 

# old    
# # determining all the connected diffs and determining their (this can be improved a lot)
# for key in diff_array:
#     if diff_array[key].type == 's':
#         for k, ins in instances.items():
#             if ins[0] == key:
#                 if diff_array[ins[1]].type == '':
#                     diff_array[ins[1]].type = 'd'
#             if ins[1] == key:
#                 if diff_array[ins[0]].type == '':
#                     diff_array[ins[0]].type = 'd'
#     if diff_array[key].type == 'd':
#         for k, ins in instances.items():
#             if ins[0] == key:
#                 if diff_array[ins[1]].type == '':
#                     diff_array[ins[1]].type = 's'
#             if ins[1] == key:
#                 if diff_array[ins[0]].type == '':
#                     diff_array[ins[0]].type = 's'
#    

# ----------------------------------------------------------
for key, df in diff_array.items():
    print(key,"\t",df.type,"\t",df.side,"\t", df.net)
# ---------------------------------------------------------- 

In [48]:
# identifying parallel instances using the net names
parallel_instances = []
inst_index = list(instances.keys())
inst_visit = [False for _ in range(len(inst_index))]
for i in range(len(inst_index)):
    if i == len(inst_index) - 1 and inst_visit[inst_index[i]] == False:
        parallel_instances.append({inst_index[i]})
        break
    group = set()
    group.add(inst_index[i])
    if inst_visit[inst_index[i]] == False:
        for j in range(i + 1, len(inst_index)):
            diffs_i = instances[inst_index[i]]
            diffs_j = instances[inst_index[j]]
            nets_i = {diff_array[diffs_i[0]].net, diff_array[diffs_i[1]].net}
            nets_j = {diff_array[diffs_j[0]].net, diff_array[diffs_j[1]].net}
            
            if nets_i == nets_j:
                group.add(inst_index[j])
                inst_visit[inst_index[j]] = True
        parallel_instances.append(group)
        
 # ----------------------------------------------------------
print(parallel_instances)
# ----------------------------------------------------------        

In [46]:
# giving actual names (replacing the id's) for the diffusions and polys
instance_names = []
for n in nets:
    for poly in n.polys:
        for i, groups in enumerate(parallel_instances):
            if poly.id in groups:
                groups_list = list(groups)
                name = "MN" + str(i) + "@" + str(groups_list.index(poly.id))
                poly.id = name
                instance_names.append(poly.id)
for n in nets:
    for diff in n.diffs:
        new_diff_id = []
        if diff.id in diff_array:
            for inst in diff_array[diff.id].instances:
                for i, groups in enumerate(parallel_instances):
                    if inst in groups:
                        groups_list = list(groups)
                        inst_name = "MN" + str(i) + "@" + str(groups_list.index(inst))
                        new_diff_id.append(inst_name + ":" + diff_array[diff.id].type)
        diff.id = new_diff_id
def divide_list(original_list, n):
    chunk_size = len(original_list) // n
    chunks = [original_list[i:i + chunk_size] for i in range(0, len(original_list), chunk_size)]
    if len(chunks) > n:
        last_chunk = chunks.pop()  # Remove the last chunk
        chunks[-1].extend(last_chunk)  # Append it to the previous chunk
    return chunks
h_counter = 0
for n in nets:
    for d in n.diffs:
        if d.orientation == "h":
            h_counter += 1
nm = divide_list(instance_names, h_counter)
counter = 0
for n in nets:
    for d in n.diffs:
        if d.orientation == "h":
            lst = []
            for i in range(len(nm[counter])):
                lst.append(nm[counter][i] + ":b")
            d.id = lst
            counter += 1
# return instances, diff_array

instances, diff_array = extract_properties_improved(polygons, nets)

# print(instances)
# for _, d in diff_array.items():
#     print(f"net: {d.net} ")
#     print(f"instances: {d.instances}")
#     print(f"side: {d.side}\n")
# 
# print(instances)
# for n in nets:
#     for p in n.polys:
#         print(p.id, end=" ")
# 
# print("\n")
# for key, df in diff_array.items():
#     # print(key, df.net)
#     print(key, df.instances, df.type, df.side)
    
    
    
for n in nets:
    for p in n.polys:
        print(p.id)

for n in nets:
    for d in n.diffs:
        if d.orientation == "v":
            print(d.id)


In [20]:

label = target_net.label.text
Resistors = []
Capacitors = []
for i in range(len(nds)):
    for j in range(len(nds[i][1])):
        ni = nodes[nds[i][0]]
        nf = nodes[nds[i][1][j]]
        dist = ((ni["p"][0] - nf["p"][0]) ** 2 + (ni["p"][1] - nf["p"][1]) ** 2) ** 0.5
        if ni["t"] == "L":
            r = [label, label + ":" + str(nds[i][1][j]), round(dist * 0.028, 3)]
            Resistors.append(r)
        elif ni["t"] == 35 or type(ni["t"]) == list:
            r1 = [label + ":" + str(nds[i][0]), label + ":" + str(nds[i][1][j]), round(dist * 0.028, 3)]
            c = [label + ":" + str(nds[i][0]), "0", dist * 210e-18, 3]
            Resistors.append(r1)
            Capacitors.append(c)
            # findng the diffusion name(id)
            for df in target_net.diffs:
                if (is_point_inside_the_polygon(ni["p"], df) and df.orientation == "v"):
                    if len(df.id) > 1:
                        diff_name = df.id[0]
                    else:
                        diff_name = df.id[0]
                    r2 = [label + ":" + str(nds[i][0]), diff_name, 14]
                    Resistors.append(r2)
                    break
                elif (is_point_inside_the_polygon(ni["p"], df) and df.orientation == "h"):
                    diff_name = df.id[0]
                    r2 = [label + ":" + str(nds[i][0]), diff_name, 14]
                    Resistors.append(r2)
                    break
            # findng the polys name/id
            for cb in target_net.cbs:  # only one cb per net
                if (is_point_inside_the_polygon(ni["p"], cb)):
                    for poly in target_net.polys:
                        poly_name = poly.id[0]
                        r2 = [label + ":" + str(nds[i][0]), poly_name + ":" + "g", 342]
                        Resistors.append(r2)
                    break
        elif ni["t"] == 16:
            r2 = [label + ":" + str(nds[i][0]), label + ":" + str(nds[i][1][j]), 14 + round(dist * 0.028, 3)]
            Resistors.append(r2)
            c = [label + ":" + str(nds[i][0]), "0", dist * 210e-18, 3]
            Capacitors.append(c)
        else:
            r = [label + ":" + str(nds[i][0]), label + ":" + str(nds[i][1][j]), round(dist * 0.028, 3)]
            c = [label + ":" + str(nds[i][0]), "0", dist * 210e-18, 3]
            Resistors.append(r)
            Capacitors.append(c)

for df in target_net.diffs:
    if len(df.id) > 1:
        if df.orientation == "v":
            r = [str(df.id[0]), str(df.id[1]), 0.01]
            Resistors.append(r)

        if df.orientation == "h":
            for i in range(len(df.id) - 1):
                r = [str(df.id[i]), str(df.id[i + 1]), 0.01]
                Resistors.append(r)

p = net_parasitics(label, Resistors, Capacitors)
for i, r in enumerate(p.res_list):
    print(i, r)
    
    
# parasitics = []
# for net in nets:
#     pi, instance = extract_layout(net, nets, polygons, a1, img_output = False)
#     parasitics.append(pi)

